In [1]:
import os
import pandas as pd
import numpy as np
from binance.client import Client
from datetime import datetime ,timedelta
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
client=Client(api_key=os.environ.get('api_key'),api_secret=os.environ.get('secret_key'),tld="com")

In [3]:
def get_history(symbol, interval, start, end=None):
    # Fetch historical data using Binance API
    bars = client.futures_klines(
        symbol=symbol, interval=interval, start_str=start, end_str=end,# limit=1000
    )
    
    # Create DataFrame from the raw data
    df = pd.DataFrame(bars)
    
    # Rename columns for clarity
    df.columns = [
        "open Time", "open", "high", "low", "close", "volume", "close Time",
        "Quote Asset volume", "No. of Trades", "Taker Buy Base Asset volume",
        "Taker Buy Quote Asset volume", "Ignore"
    ]
    
    # Convert 'open Time' to datetime including time
    df["datetime"] = pd.to_datetime(df["open Time"], unit="ms")
    
    # Select relevant columns and set 'datetime' as the index
    df = df[["datetime", "open", "high", "low", "close", "volume"]].copy()
    df.set_index("datetime", inplace=True)
    
    # Convert columns to numeric for accurate processing
    for column in ["open", "high", "low", "close", "volume"]:
        df[column] = pd.to_numeric(df[column], errors="coerce")
    
    # Format index to display full datetime
    df.index = df.index.strftime('%Y-%m-%d %H:%M:%S')
    
    return df




In [4]:
def get_usdt_futures_symbols(client):
    exchange_info = client.futures_exchange_info()  # for USD-M Futures
    symbols = [
        s['symbol']
        for s in exchange_info['symbols']
        if s['contractType'] == 'PERPETUAL' and s['quoteAsset'] == 'USDT' and s['status'] == 'TRADING'
    ]
    return symbols

symbols = get_usdt_futures_symbols(client)
print(f"Total USDT perpetual symbols: {len(symbols)}")
print(symbols[:-1])



Total USDT perpetual symbols: 441
['BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'XRPUSDT', 'LTCUSDT', 'TRXUSDT', 'ETCUSDT', 'LINKUSDT', 'XLMUSDT', 'ADAUSDT', 'XMRUSDT', 'DASHUSDT', 'ZECUSDT', 'XTZUSDT', 'BNBUSDT', 'ATOMUSDT', 'ONTUSDT', 'IOTAUSDT', 'BATUSDT', 'VETUSDT', 'NEOUSDT', 'QTUMUSDT', 'IOSTUSDT', 'THETAUSDT', 'ALGOUSDT', 'ZILUSDT', 'KNCUSDT', 'ZRXUSDT', 'COMPUSDT', 'DOGEUSDT', 'SXPUSDT', 'KAVAUSDT', 'BANDUSDT', 'RLCUSDT', 'MKRUSDT', 'SNXUSDT', 'DOTUSDT', 'DEFIUSDT', 'YFIUSDT', 'CRVUSDT', 'TRBUSDT', 'RUNEUSDT', 'SUSHIUSDT', 'EGLDUSDT', 'SOLUSDT', 'ICXUSDT', 'STORJUSDT', 'UNIUSDT', 'AVAXUSDT', 'ENJUSDT', 'FLMUSDT', 'KSMUSDT', 'NEARUSDT', 'AAVEUSDT', 'FILUSDT', 'RSRUSDT', 'LRCUSDT', 'BELUSDT', 'AXSUSDT', 'ALPHAUSDT', 'ZENUSDT', 'SKLUSDT', 'GRTUSDT', '1INCHUSDT', 'CHZUSDT', 'SANDUSDT', 'ANKRUSDT', 'RVNUSDT', 'SFPUSDT', 'COTIUSDT', 'CHRUSDT', 'MANAUSDT', 'ALICEUSDT', 'HBARUSDT', 'ONEUSDT', 'DENTUSDT', 'CELRUSDT', 'HOTUSDT', 'MTLUSDT', 'OGNUSDT', 'NKNUSDT', '1000SHIBUSDT', 'BAKEUSDT', 'GTCUSDT',

In [5]:
# Make a data directory
os.makedirs("data", exist_ok=True)

# Get date strings
end_date = datetime.now() - timedelta(days=1)  # yesterday
start_date = end_date - timedelta(days=365)

start_str = start_date.strftime('%Y-%m-%d')
end_str = end_date.strftime('%Y-%m-%d')

In [6]:
symbols = [
    s['symbol'] for s in client.futures_exchange_info()['symbols']
    if s['contractType'] == 'PERPETUAL' and s['quoteAsset'] == 'USDT' and s['status'] == 'TRADING'
]

for symbol in symbols:
    try:
        df = get_history(symbol, "1d", start=start_str, end=end_str)
        if df.empty or len(df) < 10:
            print(f"[SKIP] {symbol}: Not enough data")
            continue
        df.to_csv(f"data/{symbol}.csv")
        # print(f"[OK] Saved: {symbol}")
    except Exception as e:
        print(f"[ERROR] {symbol}: {e}")


[SKIP] DMCUSDT: Not enough data
[SKIP] HUSDT: Not enough data
[SKIP] OLUSDT: Not enough data
[SKIP] SAHARAUSDT: Not enough data


In [7]:

# Set the data directory path (adjust as needed)
data_path = "data/"

# List all CSV files in the folder
files = [f for f in os.listdir(data_path) if f.endswith(".csv")]

# Setup date range: last 7 days ending yesterday
today = datetime.now().date()
date_list = [today - timedelta(days=i) for i in range(1, 8)]
date_list.sort()

returns_data = {}

# Process each file
for file in files:
    symbol = file.replace(".csv", "")
    df = pd.read_csv(os.path.join(data_path, file), parse_dates=["datetime"])
    df.set_index("datetime", inplace=True)
    df = df.sort_index()
    
    df["log_return"] = np.log(df["close"] / df["close"].shift(1))
    df["pct_return"] = df["log_return"] * 100
    df["date"] = df.index.date
    
    # Get returns for the last 7 days
    row = []
    for date in date_list:
        val = df[df["date"] == date]["pct_return"]
        row.append(val.iloc[0] if not val.empty else np.nan)
    
    returns_data[symbol] = row

# Create DataFrame
heatmap_df = pd.DataFrame(returns_data, index=[d.strftime('%Y-%m-%d') for d in date_list]).T
heatmap_df.columns = [f"D{i+1}" for i in range(7)]  # Label columns as D1 to D7

# Plot scrollable heatmap using Plotly
fig = px.imshow(
    heatmap_df,
    text_auto=".2f",
    color_continuous_scale="RdYlGn",
    labels=dict(x="Day", y="Asset", color="% Return"),
    aspect="auto",
    title="7-Day Coin Performance Heatmap",
    zmin=-15,  # Set the minimum value for color scale
    zmax=15    # Set the maximum value for color scale
)
fig.update_layout(
    template="plotly_dark",
    height=16000,  # Enough height to show boxes clearly
    width=1000,
    yaxis_nticks=len(heatmap_df),
)
fig.update_yaxes(tickfont=dict(size=10), automargin=True)
fig.show()

In [8]:
# # --- Setup Weekly and Monthly Ranges ---

# # Get last 5 completed weeks (Monday to Sunday)
# week_ends = []
# for i in range(1, 6):
#     end_of_week = today - timedelta(days=today.weekday() + 1 + (7 * (i - 1)))  # Last Sunday
#     start_of_week = end_of_week - timedelta(days=6)
#     week_ends.append((start_of_week, end_of_week))

# # Get last 7 completed months
# month_ranges = []
# for i in range(1, 8):
#     month = (today.replace(day=1) - pd.DateOffset(months=i)).date()
#     start_of_month = month
#     end_of_month = (month.replace(day=1) + pd.DateOffset(months=1) - pd.DateOffset(days=1)).date()
#     month_ranges.append((start_of_month, end_of_month))

# # --- Function to calculate period returns ---
# def compute_period_returns(ranges, label_prefix):
#     period_data = {}
    
#     for file in files:
#         symbol = file.replace(".csv", "")
#         df = pd.read_csv(os.path.join(data_path, file), parse_dates=["datetime"])
#         df.set_index("datetime", inplace=True)
#         df = df.sort_index()
#         df["close"] = pd.to_numeric(df["close"], errors="coerce")
#         df["date"] = df.index.date

#         row = []
#         for start_date, end_date in ranges:
#             df_period = df[(df["date"] >= start_date) & (df["date"] <= end_date)]
#             if len(df_period) >= 2:
#                 start_price = df_period["close"].iloc[0]
#                 end_price = df_period["close"].iloc[-1]
#                 ret = np.log(end_price / start_price) * 100
#                 row.append(ret)
#             else:
#                 row.append(np.nan)
#         period_data[symbol] = row

#     col_labels = [f"{label_prefix}{i+1}" for i in range(len(ranges))]
#     return pd.DataFrame(period_data, index=col_labels).T

# # --- Compute Weekly Returns ---
# weekly_df = compute_period_returns(week_ends, "W")

# # --- Compute Monthly Returns ---
# monthly_df = compute_period_returns(month_ranges, "M")

# # --- Plot Weekly Heatmap ---
# fig_w = px.imshow(
#     weekly_df,
#     text_auto=".2f",
#     color_continuous_scale="RdYlGn",
#     labels=dict(x="Week", y="Asset", color="% Return"),
#     title="5-Week Coin Performance Heatmap",
#     zmin=-50,
#     zmax=50,
#     aspect="auto"
# )
# fig_w.update_layout(template="plotly_dark", height=16000, width=1000, yaxis_nticks=len(weekly_df))
# fig_w.update_yaxes(tickfont=dict(size=10), automargin=True)
# fig_w.show()

# # --- Plot Monthly Heatmap ---
# fig_m = px.imshow(
#     monthly_df,
#     text_auto=".2f",
#     color_continuous_scale="RdYlGn",
#     labels=dict(x="Month", y="Asset", color="% Return"),
#     title="7-Month Coin Performance Heatmap",
#     zmin=-100,
#     zmax=100,
#     aspect="auto"
# )
# fig_m.update_layout(template="plotly_dark", height=16000, width=1000, yaxis_nticks=len(monthly_df))
# fig_m.update_yaxes(tickfont=dict(size=10), automargin=True)
# fig_m.show()


In [9]:
from IPython.display import display, HTML

# Set the data directory path
data_path = "data/"
os.makedirs(data_path, exist_ok=True)

# List all CSV files in the folder
files = [f for f in os.listdir(data_path) if f.endswith(".csv")]

# Setup date range: last 7 days ending yesterday
today = datetime.now().date()
date_list = [today - timedelta(days=i) for i in range(1, 8)]
date_list.sort()

# Data structures to store all metrics
returns_data = {}
volume_data = {}
daily_metrics = {date: {'gainers': [], 'losers': [], 'volume_leaders': []} for date in date_list}

# Process each file
for file in files:
    symbol = file.replace(".csv", "")
    try:
        df = pd.read_csv(os.path.join(data_path, file), parse_dates=["datetime"])
        df.set_index("datetime", inplace=True)
        df = df.sort_index()
        
        # Calculate returns
        df["log_return"] = np.log(df["close"] / df["close"].shift(1))
        df["pct_return"] = df["log_return"] * 100
        df["date"] = df.index.date
        
        # Get daily metrics
        for date in date_list:
            daily_df = df[df["date"] == date]
            if not daily_df.empty:
                # Get return for heatmap
                returns_data.setdefault(symbol, []).append(daily_df["pct_return"].iloc[0])
                
                # Calculate volume in USDT terms (volume * closing price)
                volume_usdt = daily_df["volume"].sum() * daily_df["close"].iloc[-1]
                volume_data.setdefault(symbol, []).append(volume_usdt)
                
                # Store metrics for daily analysis
                daily_metrics[date]['gainers'].append((symbol, daily_df["pct_return"].iloc[0]))
                daily_metrics[date]['losers'].append((symbol, daily_df["pct_return"].iloc[0]))
                daily_metrics[date]['volume_leaders'].append((symbol, volume_usdt))
    except Exception as e:
        print(f"Error processing {file}: {e}")

# Generate daily reports
for date in date_list:
    date_str = date.strftime('%Y-%m-%d')
    
    # Get top gainers/losers
    gainers = sorted(daily_metrics[date]['gainers'], key=lambda x: x[1], reverse=True)[:10]
    losers = sorted(daily_metrics[date]['losers'], key=lambda x: x[1])[:10]
    volume_leaders = sorted(daily_metrics[date]['volume_leaders'], key=lambda x: x[1], reverse=True)[:10]
    
    # Create DataFrames for display
    gainers_df = pd.DataFrame(gainers, columns=['Symbol', 'Return (%)'])
    losers_df = pd.DataFrame(losers, columns=['Symbol', 'Return (%)'])
    volume_df = pd.DataFrame(volume_leaders, columns=['Symbol', 'Volume (USDT)'])
    
    # # Display daily report
    # display(HTML(f"<h2>Daily Report - {date_str}</h2>"))
    # display(HTML("<h3>Top 10 Gainers</h3>"))
    # display(gainers_df.style.format({'Return (%)': '{:.2f}%'}))
    
    # display(HTML("<h3>Top 10 Losers</h3>"))
    # display(losers_df.style.format({'Return (%)': '{:.2f}%'}))
    
    # display(HTML("<h3>Top 10 Volume Leaders (USDT)</h3>"))
    # display(volume_df.style.format({'Volume (USDT)': '${:,.0f}'}))

# Generate weekly summary
weekly_gainers = []
weekly_losers = []
weekly_volume = []

for symbol in returns_data:
    weekly_return = sum(returns_data[symbol])
    weekly_volume_sum = sum(volume_data.get(symbol, [0]))
    weekly_gainers.append((symbol, weekly_return))
    weekly_losers.append((symbol, weekly_return))
    weekly_volume.append((symbol, weekly_volume_sum))

# Get top weekly performers
top_weekly_gainers = sorted(weekly_gainers, key=lambda x: x[1], reverse=True)[:10]
top_weekly_losers = sorted(weekly_losers, key=lambda x: x[1])[:10]
top_weekly_volume = sorted(weekly_volume, key=lambda x: x[1], reverse=True)[:50]

# Create DataFrames for weekly summary
weekly_gainers_df = pd.DataFrame(top_weekly_gainers, columns=['Symbol', 'Weekly Return (%)'])
weekly_losers_df = pd.DataFrame(top_weekly_losers, columns=['Symbol', 'Weekly Return (%)'])
weekly_volume_df = pd.DataFrame(top_weekly_volume, columns=['Symbol', 'Weekly Volume (USDT)'])

# # Display weekly summary
# display(HTML("<h2>Weekly Summary</h2>"))
# display(HTML("<h3>Top 10 Weekly Gainers</h3>"))
# display(weekly_gainers_df.style.format({'Weekly Return (%)': '{:.2f}%'}))

# display(HTML("<h3>Top 10 Weekly Losers</h3>"))
# display(weekly_losers_df.style.format({'Weekly Return (%)': '{:.2f}%'}))

# display(HTML("<h3>Top 10 Weekly Volume Leaders (USDT)</h3>"))
# display(weekly_volume_df.style.format({'Weekly Volume (USDT)': '${:,.0f}'}))

In [10]:


# 2. Top Movers Dashboard
def create_movers_figure(date):
    gainers = sorted(daily_metrics[date]['gainers'], key=lambda x: x[1], reverse=True)[:10]
    losers = sorted(daily_metrics[date]['losers'], key=lambda x: x[1])[:10]
    
    fig = make_subplots(rows=1, cols=2, subplot_titles=(f"Top Gainers - {date}", f"Top Losers - {date}"))
    
    fig.add_trace(go.Bar(
        x=[x[0] for x in gainers],
        y=[x[1] for x in gainers],
        marker_color='green',
        name='Gainers'
    ), row=1, col=1)
    
    fig.add_trace(go.Bar(
        x=[x[0] for x in losers],
        y=[x[1] for x in losers],
        marker_color='red',
        name='Losers'
    ), row=1, col=2)
    
    fig.update_layout(
        showlegend=False,
        height=500,
        template="plotly_dark",
        yaxis=dict(title="Return %"),
        yaxis2=dict(title="Return %")
    )
    
    return fig

# Display movers for each day
for date in date_list[-3:]:  # Show last 3 days for brevity
    create_movers_figure(date).show()


# 4. Weekly Performance Scatter Plot
weekly_df = pd.DataFrame({
    'Symbol': [x[0] for x in weekly_gainers],
    'Return': [x[1] for x in weekly_gainers],
    'Volume': [sum(volume_data.get(x[0], [0])) for x in weekly_gainers]
})

fig_scatter = px.scatter(
    weekly_df,
    x='Volume',
    y='Return',
    color='Return',
    size='Volume',
    hover_name='Symbol',
    log_x=True,
    title='Weekly Performance vs Trading Volume',
    labels={'Return': 'Weekly Return %', 'Volume': 'Trading Volume (USDT)'},
    color_continuous_scale='RdYlGn'
)

fig_scatter.update_layout(
    height=600,
    width=1000,
    template="plotly_dark",
    hovermode='closest'
)
fig_scatter.show()

# 5. Cumulative Returns Line Chart
top_symbols = [x[0] for x in weekly_gainers[:10]] + [x[0] for x in weekly_losers[:10]]

fig_lines = go.Figure()
for symbol in top_symbols:
    fig_lines.add_trace(go.Scatter(
        x=date_list,
        y=np.cumsum(returns_data.get(symbol, [0]*7)),
        name=symbol,
        mode='lines+markers'
    ))

fig_lines.update_layout(
    title='Cumulative Returns of Top Performers',
    yaxis=dict(title='Cumulative Return %'),
    xaxis=dict(title='Date'),
    height=600,
    width=1000,
    template="plotly_dark",
    hovermode='x unified'
)
fig_lines.show()

In [11]:
# Prepare combined performance and volume data
treemap_data = []
for symbol in returns_data.keys():
    weekly_return = sum(returns_data[symbol])
    weekly_volume = sum(volume_data.get(symbol, [0]))
    treemap_data.append({
        'Symbol': symbol,
        'USDT Volume': weekly_volume,
        'Weekly Return': weekly_return,
        'Performance': 'Positive' if weekly_return >= 0 else 'Negative'
    })

treemap_df = pd.DataFrame(treemap_data)

# Create the treemap
fig_volume = px.treemap(
    treemap_df,
    path=['Symbol'],
    values='USDT Volume',
    title='All Cryptocurrencies by Trading Volume (USDT) - Colored by Weekly Performance',
    color='Performance',
    color_discrete_map={'Positive':'#3D9970', 'Negative':'#FF4136'},  # Green and red
    hover_data=['Weekly Return', 'USDT Volume'],
    hover_name='Symbol',
    width=1200,
    height=800
)

# Format hover template
fig_volume.update_traces(
    hovertemplate="<b>%{label}</b><br>" +
                 "Volume: %{value:,.0f} USDT<br>" +
                 "Weekly Return: %{customdata[0]:.2f}%<extra></extra>"
)

# Customize layout
fig_volume.update_layout(
    template="plotly_dark",
    margin=dict(t=80, l=25, r=25, b=25),
    uniformtext=dict(minsize=12, mode='hide'),
    coloraxis_showscale=False
)

# Add annotation about color meaning

# Customize layout
fig_volume.update_layout(
    template="plotly_dark",
    margin=dict(t=80, l=25, r=25, b=25),
    uniformtext=dict(minsize=10, mode='hide'),
    annotations=[
        dict(
            x=0.5, y=1.05,
            xref="paper", yref="paper",
            text="Color indicates daily performance: <span style='color:#3D9970'>Green = Positive</span>, <span style='color:#FF4136'>Red = Negative</span>",
            showarrow=False,
            font=dict(size=12)
        )
    ]
)

fig_volume.show()

In [12]:
# Get yesterday's date
yesterday = datetime.now().date() - timedelta(days=1)

# Prepare yesterday's data
treemap_data = []
for symbol in returns_data.keys():
    # Find yesterday's index in date_list
    if yesterday in date_list:
        idx = date_list.index(yesterday)
        daily_return = returns_data[symbol][idx] if idx < len(returns_data[symbol]) else 0
        daily_volume = volume_data.get(symbol, [0]*7)[idx] if idx < len(volume_data.get(symbol, [])) else 0
        
        treemap_data.append({
            'Symbol': symbol,
            'USDT Volume': daily_volume,
            'Daily Return': daily_return,
            'Performance': 'Positive' if daily_return >= 0 else 'Negative'
        })

treemap_df = pd.DataFrame(treemap_data)

# Create the treemap
fig_volume = px.treemap(
    treemap_df,
    path=['Symbol'],
    values='USDT Volume',
    title=f'Cryptocurrencies by Trading Volume (USDT) - {yesterday.strftime("%Y-%m-%d")}',
    color='Performance',
    color_discrete_map={'Positive':'#3D9970', 'Negative':'#FF4136'},
    hover_data=['Daily Return', 'USDT Volume'],
    width=1200,
    height=800
)

# Format hover template
fig_volume.update_traces(
    hovertemplate="<b>%{label}</b><br>" +
                 "Volume: $%{value:,.0f}<br>" +
                 "Daily Return: %{customdata[0]:.2f}%<extra></extra>"
)

# Customize layout
fig_volume.update_layout(
    template="plotly_dark",
    margin=dict(t=80, l=25, r=25, b=25),
    uniformtext=dict(minsize=10, mode='hide'),
    annotations=[
        dict(
            x=0.5, y=1.05,
            xref="paper", yref="paper",
            text="Color indicates daily performance: <span style='color:#3D9970'>Green = Positive</span>, <span style='color:#FF4136'>Red = Negative</span>",
            showarrow=False,
            font=dict(size=12)
        )
    ]
)

fig_volume.show()

In [13]:
# Calculate market-wide metrics
total_volume = sum([sum(v) for v in volume_data.values()])
avg_daily_return = heatmap_df.mean().mean()
positive_days = (heatmap_df > 0).sum().sum()
market_metrics = {
    'Total Weekly Volume (USDT)': f"${total_volume:,.0f}",
    'Avg Daily Return %': f"{avg_daily_return:.2f}%",
    'Positive Days %': f"{(positive_days/(heatmap_df.size))*100:.1f}%",
    'Top Gainer': weekly_gainers_df.iloc[0]['Symbol'],
    'Top Gainer Return': f"{weekly_gainers_df.iloc[0]['Weekly Return (%)']:.2f}%"
}

display(HTML("<h2>📊 Market Health Dashboard</h2>"))
display(pd.DataFrame.from_dict(market_metrics, orient='index', columns=['Value']))



,Value
Total Weekly Volume (USDT),"$328,873,933,733"
Avg Daily Return %,1.55%
Positive Days %,61.5%
Top Gainer,MYXUSDT
Top Gainer Return,55.77%


In [14]:
# Calculate correlation between top 20 coins
corr_matrix = heatmap_df[heatmap_df.index.isin(weekly_volume_df.head(100)['Symbol'])].T.corr()

fig = go.Figure(go.Heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns,
    y=corr_matrix.index,
    colorscale='RdBu',
    zmin=-1,
    zmax=1
))
fig.update_layout(title='Top 25 Coins Correlation Matrix',
                 height=800,
                 width=800)
fig.show()